# Gather TEAM and Player Roster

In [1]:
# Install Dependencies
from nba_api.stats.endpoints import playercareerstats # Provides single player career stats"
from nba_api.live.nba.endpoints import scoreboard # Provides scoreboards for that day"
from nba_api.stats.static import players # Receive player information i.e. player_id
from nba_api.stats.static import teams # Retrieve team data
from nba_api.stats.endpoints import leaguegamefinder # Retrieve specific games
from nba_api.stats.endpoints import leagueleaders # Retrieve stat leaders for a season
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Retrieve dataframe of team plus players
teams = teams.get_teams()

# Turn dictionary into a pandas dataframe
teams = pd.DataFrame.from_dict(teams)

# Show the dataframe
teams

AttributeError: 'list' object has no attribute 'get_teams'

In [ ]:
# Get player information
player_dict = players.get_players()

# Change the dict to dataframe
nba_players = pd.DataFrame.from_dict(player_dict)

# Get only active nba players
active_players = nba_players.loc[nba_players['is_active'] == True]

# Show the active players
active_players.head()

In [ ]:
# Look for a specific player
James_Harden = active_players.loc[active_players['full_name']=='James Harden']
James_Harden

In [ ]:
# Locate the Spurs team to get the team id
Rockets_id = teams.loc[teams['nickname'] == 'Rockets']

# Show the id
Rockets_id

In [ ]:
# Retrieve roster for 2022-2023
teamfinder = commonteamroster.CommonTeamRoster(season='2022-23',
                                              team_id = '1610612759',
                                              league_id_nullable = '00')

nba_teams = teamfinder.get_data_frames()[0]

team_id = nba_teams['TeamID'].unique().tolist()
team_ids = nba_teams['TeamID'].unique().tolist()

spurs_roster = []

for team_id in team_ids:
    player_stats_data = commonteamroster.CommonTeamRoster(season = '2022-23', team_id = team_id, timeout=100)
    df = player_stats_data.common_team_roster.get_data_frame()
    df['TeamID'] = team_id
    spurs_roster.append(df)
    print(spurs_roster)
#Spurs_roster = nba_commonteamroster(season = '2022-23', team_id = '1610612759')

# Gather shot chart information for specific player

In [ ]:
# Get the shot detail of a player "Devin Vassell"
shot_detail = shotchartdetail.ShotChartDetail(player_id= 1630170, 
              team_id=1610612759 , context_measure_simple = 'FGA',
              season_type_all_star='Regular Season', season_nullable='2022-23')#.get_data_frames()[0]

# Turn the data into a dataframe
#shot_df = shot_detail

# Show the dataframe
shot_df

In [ ]:
# Check the dtypes
shot_df.dtypes

In [ ]:
# View the unique values for SHOT_ZONE_BASIC
shot_df.SHOT_ZONE_BASIC.unique()

In [ ]:
# View unique values
shot_df.SHOT_ZONE_AREA.unique()

In [ ]:
#View unique values
shot_df.ACTION_TYPE.unique()

In [ ]:
# View unique values
shot_df.SHOT_ZONE_RANGE.unique()

In [ ]:
# View unique values
shot_df.SHOT_TYPE.unique()

In [ ]:
# Create a shots made dataframe
shots_df = shot_df [['PLAYER_NAME', 'TEAM_NAME', 'SHOT_ZONE_BASIC', 'EVENT_TYPE','LOC_X', 
                       'LOC_Y', 'SHOT_TYPE', 'ACTION_TYPE', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE']]

# Show the made shot
shots_df

# Create Shot Chart

In [ ]:
# Create the basketball chart using formula from
from matplotlib.patches import Circle, Rectangle, Arc

def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [ ]:
# Show the court
plt.figure(figsize = (12,11))
draw_court(outer_lines = True, color = "black")
plt.xlim(-300,300)
plt.ylim(-100,500)
plt.show()

In [ ]:
# Create a scatter plot of the shots by player
plt.figure(figsize = (12,11))
plt.scatter(shot_df.LOC_X, shot_df.LOC_Y, c=shot_df.SHOT_MADE_FLAG, cmap='Greens')
draw_court(outer_lines = True)
# Descending values along the y axis from bottom to top in order to place the hoop by the top of the plot
plt.ylim(422.5, -47.5)
plt.show()

In [ ]:
# Compare to the hexbin
shots_hex = plt.hexbin(
    shots_df.LOC_X, shots_df.LOC_Y,
    extent=(-250, 250, 422.5, -47.5), cmap='Blues', gridsize=40)
plt.close()  # this closes the plot window

makes_df = shots_df[shots_df.EVENT_TYPE == 'Made Shot']
makes_hex = plt.hexbin(
    makes_df['LOC_X'], makes_df['LOC_Y'],
    extent=(-250, 250, 422.5, -47.5), cmap=plt.cm.Reds, gridsize=40)
plt.close()
pcts_by_hex = makes_hex.get_array() / shots_hex.get_array()
pcts_by_hex[np.isnan(pcts_by_hex)] = 0  # convert NAN values to 0

sample_sizes = shots_hex.get_array()
filter_threshold = 5
for i in range(len(pcts_by_hex)):
    if sample_sizes[i] < filter_threshold:
        pcts_by_hex[i] = 0
x = [i[0] for i in shots_hex.get_offsets()]
y = [i[1] for i in shots_hex.get_offsets()]
z = pcts_by_hex

plt.figure(figsize=(5, 4.7))
plt.xlim(250, -250)
plt.ylim(-47.5, 422.5)
plt.scatter(x, y, c=z, cmap='Blues', marker='h')
plt.text(220, 350, "Shot chart: \nKeldon Johnson ('22-'23)")
draw_court(outer_lines=True)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)

In [ ]:
shots_hex = plt.hexbin(
    shots_df.LOC_X, shots_df.LOC_Y,
    extent=(-250, 250, 422.5, -47.5), cmap=plt.cm.Reds, gridsize=25)
shots_by_hex = shots_hex.get_array()
freq_by_hex = shots_by_hex / sum(shots_by_hex)
sizes = freq_by_hex / max(freq_by_hex) * 120
plt.close()

plt.figure(figsize=(5, 4.7))
plt.xlim(250, -250)
plt.ylim(-47.5, 422.5)

plt.scatter(x, y, c=z, cmap='Blues', marker='h')
plt.text(220, 350, "Shot chart: \nJames Harden ('18-'19)")
draw_court(outer_lines=True)
#cur_axes = plt.gca()
#cur_axes.axes.get_xaxis().set_visible(False)
#cur_axes.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Save chart
plt.savefig('Shot Chart/Keldon_Johnson_Shot_Chart.png')

# Scatter Plot

In [ ]:
# Get all nba players for the 22-23 season in order by best ranked,
# Make the total number large enough to make sure every player gets added
top_1000 = leagueleaders.LeagueLeaders(season = '2022-23', season_type_all_star = 'Regular Season',
                                     stat_category_abbreviation = 'PTS').get_data_frames()[0][:1000]

# Show the dataframe
top_1000.head()

In [ ]:
# Get the number of total players
top_1000.count()

In [ ]:
# Get the Spurs Roster for 22-23
spurs_roster = top_1000.loc[top_1000['TEAM'] == 'SAS']

# Show the roster
spurs_roster

In [ ]:
# Create Scatter Plots
# Create the x axis
x_axis=spurs_roster.PTS

# Create the y axis
y_axis=spurs_roster.FG3M/spurs_roster.FG3A